## Лаба 5: Прогнозирование оттока клиентов

In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 --executor-memory 3g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark.sql.functions import json_tuple, from_json, get_json_object, col, explode, expr, \
collect_set, collect_list,regexp_replace, get_json_object, to_json, struct
import pyspark.sql.types as t
import pyspark.sql.functions as f

from pyspark.sql.types import StructField, StructType, StringType, ArrayType, IntegerType, TimestampType, LongType
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, StringIndexer, IndexToString
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.linalg import Vector, DenseVector
from pyspark.ml.classification import LogisticRegression ,DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import json, pprint

In [3]:
!hdfs dfs -ls /labs/slaba05/

Found 2 items
-rw-r--r--   3 hdfs hdfs   24271415 2022-01-06 18:46 /labs/slaba05/lab05_test.csv
-rw-r--r--   3 hdfs hdfs  176160768 2022-01-06 18:46 /labs/slaba05/lab05_train.csv


In [4]:
!hdfs dfs -cat /labs/slaba05/lab05_train.csv | head -n 2

,ID,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,PRC_ACCEPTS_A_EMAIL_LINK,APP_REGISTR_RGN_CODE,PRC_ACCEPTS_A_POS,PRC_ACCEPTS_A_TK,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,PRC_ACCEPTS_A_AMOBILE,SUM_TRAN_AUT_TENDENCY3M,CLNT_TRUST_RELATION,PRC_ACCEPTS_TK,PRC_ACCEPTS_A_MTP,REST_DYNAMIC_FDEP_1M,CNT_TRAN_AUT_TENDENCY3M,CNT_ACCEPTS_TK,APP_MARITAL_STATUS,REST_DYNAMIC_SAVE_3M,CR_PROD_CNT_VCU,REST_AVG_CUR,CNT_TRAN_MED_TENDENCY1M,APP_KIND_OF_PROP_HABITATION,CLNT_JOB_POSITION_TYPE,AMOUNT_RUB_NAS_PRC,CLNT_JOB_POSITION,APP_DRIVING_LICENSE,TRANS_COUNT_SUP_PRC,APP_EDUCATION,CNT_TRAN_CLO_TENDENCY1M,SUM_TRAN_MED_TENDENCY1M,PRC_ACCEPTS_A_ATM,PRC_ACCEPTS_MTP,TRANS_COUNT_NAS_PRC,APP_TRAVEL_PASS,CNT_ACCEPTS_MTP,CR_PROD_CNT_TOVR,APP_CAR,CR_PROD_CNT_PIL,SUM_TRAN_CLO_TENDENCY1M,APP_POSITION_TYPE,TURNOVER_CC,TRANS_COUNT_ATM_PRC,AMOUNT_RUB_ATM_PRC,TURNOVER_PAYM,AGE,CNT_TRAN_MED_TENDENCY3M,CR_PROD_CNT_CC,SUM_TRAN_MED_TENDENCY3M,REST_DYNAMIC_FDEP_3M,REST_DYNAMIC_IL_1M,APP_EMP_

In [5]:
!hdfs dfs -cat /labs/slaba05/lab05_test.csv | head -n 2

,ID,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,PRC_ACCEPTS_A_EMAIL_LINK,APP_REGISTR_RGN_CODE,PRC_ACCEPTS_A_POS,PRC_ACCEPTS_A_TK,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,PRC_ACCEPTS_A_AMOBILE,SUM_TRAN_AUT_TENDENCY3M,CLNT_TRUST_RELATION,PRC_ACCEPTS_TK,PRC_ACCEPTS_A_MTP,REST_DYNAMIC_FDEP_1M,CNT_TRAN_AUT_TENDENCY3M,CNT_ACCEPTS_TK,APP_MARITAL_STATUS,REST_DYNAMIC_SAVE_3M,CR_PROD_CNT_VCU,REST_AVG_CUR,CNT_TRAN_MED_TENDENCY1M,APP_KIND_OF_PROP_HABITATION,CLNT_JOB_POSITION_TYPE,AMOUNT_RUB_NAS_PRC,CLNT_JOB_POSITION,APP_DRIVING_LICENSE,TRANS_COUNT_SUP_PRC,APP_EDUCATION,CNT_TRAN_CLO_TENDENCY1M,SUM_TRAN_MED_TENDENCY1M,PRC_ACCEPTS_A_ATM,PRC_ACCEPTS_MTP,TRANS_COUNT_NAS_PRC,APP_TRAVEL_PASS,CNT_ACCEPTS_MTP,CR_PROD_CNT_TOVR,APP_CAR,CR_PROD_CNT_PIL,SUM_TRAN_CLO_TENDENCY1M,APP_POSITION_TYPE,TURNOVER_CC,TRANS_COUNT_ATM_PRC,AMOUNT_RUB_ATM_PRC,TURNOVER_PAYM,AGE,CNT_TRAN_MED_TENDENCY3M,CR_PROD_CNT_CC,SUM_TRAN_MED_TENDENCY3M,REST_DYNAMIC_FDEP_3M,REST_DYNAMIC_IL_1M,APP_EMP_

In [28]:
df_train = spark.read.csv("/labs/slaba05/lab05_train.csv", header=True, inferSchema=True, sep=',')
# df_train.show(2, vertical = True, truncate = False)
# df_train.count() # 320764

320764

In [29]:
df_test = spark.read.csv("/labs/slaba05/lab05_test.csv", header=True, inferSchema=True, sep=',')
# df_test.show(2, vertical = True, truncate = False)
# df_test.count() # 44399

44399

## 1. Выбираем фичи

In [14]:
difference = list(set(df_train.columns).symmetric_difference(set(df_test.columns)))
print(difference)

['TARGET']


In [31]:
# import pyspark.sql.functions as func

# lst = [];
# for col in df_train.columns:
#     if (df_train.filter(func.col(col).isNull()).count() == df_train.select(func.col(col)).count()):
# #         df_train = df_train.drop(col)
#         lst.append(col)

In [42]:
import pyspark.sql.functions as func

lst = [];
for col in df_train.columns:
    lst.append(    df_train.filter(func.col(col).isNull()).count()    )
lst

[0,
 0,
 0,
 34550,
 180814,
 265914,
 180814,
 180814,
 1,
 251229,
 251229,
 34551,
 180814,
 220560,
 257829,
 180814,
 180814,
 1,
 220560,
 180814,
 258989,
 1,
 1,
 1,
 258307,
 267042,
 280242,
 34551,
 130371,
 268868,
 34551,
 259099,
 260941,
 258307,
 180814,
 180814,
 34551,
 268868,
 180814,
 1,
 268869,
 1,
 260941,
 265984,
 1,
 34551,
 34551,
 1,
 1,
 216119,
 1,
 216119,
 1,
 1,
 259778,
 216989,
 313405,
 319629,
 1,
 313405,
 319635,
 313405,
 216989,
 1,
 1,
 313405,
 319060,
 259778,
 1,
 1,
 141235,
 1,
 1,
 141235,
 1,
 89923,
 134853,
 1,
 89923,
 257149,
 319296,
 234276,
 179914,
 234276,
 134853,
 1,
 179914,
 258071,
 1,
 1,
 319065,
 318213,
 1,
 258071,
 1,
 313405,
 1,
 320112,
 46860,
 313405,
 46860,
 313405,
 1,
 234276,
 1,
 319060,
 236350,
 1,
 234276,
 236350,
 232072,
 234276,
 1,
 236350,
 236350,
 236350,
 1]

In [43]:
len(lst)

117

In [44]:
len(df_train.columns)

117

In [32]:
# df_train =  df_train.na.fill(-1)
# df_train.show(1, vertical = True, truncate = False)

In [33]:
# df_test = df_test.na.fill(-1)
# df_test.show(2, vertical = True, truncate = False)

In [54]:
name = [];
for col in df_train.columns:
    if df_train.filter(func.col(col).isNull()).count() < 2:
        name.append(col)
name

['_c0',
 'ID',
 'CR_PROD_CNT_IL',
 'TURNOVER_DYNAMIC_IL_1M',
 'REST_DYNAMIC_FDEP_1M',
 'REST_DYNAMIC_SAVE_3M',
 'CR_PROD_CNT_VCU',
 'REST_AVG_CUR',
 'CR_PROD_CNT_TOVR',
 'CR_PROD_CNT_PIL',
 'TURNOVER_CC',
 'TURNOVER_PAYM',
 'AGE',
 'CR_PROD_CNT_CC',
 'REST_DYNAMIC_FDEP_3M',
 'REST_DYNAMIC_IL_1M',
 'CR_PROD_CNT_CCFP',
 'REST_DYNAMIC_CUR_1M',
 'REST_AVG_PAYM',
 'LDEAL_GRACE_DAYS_PCT_MED',
 'REST_DYNAMIC_CUR_3M',
 'TURNOVER_DYNAMIC_CUR_1M',
 'REST_DYNAMIC_PAYM_3M',
 'REST_DYNAMIC_IL_3M',
 'TURNOVER_DYNAMIC_IL_3M',
 'REST_DYNAMIC_PAYM_1M',
 'TURNOVER_DYNAMIC_CUR_3M',
 'PACK',
 'CLNT_SETUP_TENOR',
 'TURNOVER_DYNAMIC_PAYM_3M',
 'TURNOVER_DYNAMIC_PAYM_1M',
 'REST_DYNAMIC_CC_1M',
 'TURNOVER_DYNAMIC_CC_1M',
 'REST_DYNAMIC_CC_3M',
 'TURNOVER_DYNAMIC_CC_3M',
 'TARGET']

In [63]:
len(name)

35

In [55]:
df_train_short = df_train.select(name)
df_train_short.show(2, vertical = True, truncate = False)

-RECORD 0---------------------------------------
 _c0                      | 333149              
 ID                       | 479990              
 CR_PROD_CNT_IL           | 0                   
 TURNOVER_DYNAMIC_IL_1M   | 0.0                 
 REST_DYNAMIC_FDEP_1M     | 0.0                 
 REST_DYNAMIC_SAVE_3M     | 0.0                 
 CR_PROD_CNT_VCU          | 0                   
 REST_AVG_CUR             | 19069.6917166667    
 CR_PROD_CNT_TOVR         | 1                   
 CR_PROD_CNT_PIL          | 0                   
 TURNOVER_CC              | 0.0                 
 TURNOVER_PAYM            | 0.0                 
 AGE                      | 324                 
 CR_PROD_CNT_CC           | 0                   
 REST_DYNAMIC_FDEP_3M     | 0.0                 
 REST_DYNAMIC_IL_1M       | 0.0                 
 CR_PROD_CNT_CCFP         | 0                   
 REST_DYNAMIC_CUR_1M      | 0.139327197286461   
 REST_AVG_PAYM            | 0.0                 
 LDEAL_GRACE_DAYS_PC

In [65]:
name

['_c0',
 'ID',
 'CR_PROD_CNT_IL',
 'TURNOVER_DYNAMIC_IL_1M',
 'REST_DYNAMIC_FDEP_1M',
 'REST_DYNAMIC_SAVE_3M',
 'CR_PROD_CNT_VCU',
 'REST_AVG_CUR',
 'CR_PROD_CNT_TOVR',
 'CR_PROD_CNT_PIL',
 'TURNOVER_CC',
 'TURNOVER_PAYM',
 'AGE',
 'CR_PROD_CNT_CC',
 'REST_DYNAMIC_FDEP_3M',
 'REST_DYNAMIC_IL_1M',
 'CR_PROD_CNT_CCFP',
 'REST_DYNAMIC_CUR_1M',
 'REST_AVG_PAYM',
 'LDEAL_GRACE_DAYS_PCT_MED',
 'REST_DYNAMIC_CUR_3M',
 'TURNOVER_DYNAMIC_CUR_1M',
 'REST_DYNAMIC_PAYM_3M',
 'REST_DYNAMIC_IL_3M',
 'TURNOVER_DYNAMIC_IL_3M',
 'REST_DYNAMIC_PAYM_1M',
 'TURNOVER_DYNAMIC_CUR_3M',
 'PACK',
 'CLNT_SETUP_TENOR',
 'TURNOVER_DYNAMIC_PAYM_3M',
 'TURNOVER_DYNAMIC_PAYM_1M',
 'REST_DYNAMIC_CC_1M',
 'TURNOVER_DYNAMIC_CC_1M',
 'REST_DYNAMIC_CC_3M',
 'TURNOVER_DYNAMIC_CC_3M',
 'TARGET']

In [66]:
name_test = name.copy()
name_test.remove("TARGET")

In [67]:
name_test

['_c0',
 'ID',
 'CR_PROD_CNT_IL',
 'TURNOVER_DYNAMIC_IL_1M',
 'REST_DYNAMIC_FDEP_1M',
 'REST_DYNAMIC_SAVE_3M',
 'CR_PROD_CNT_VCU',
 'REST_AVG_CUR',
 'CR_PROD_CNT_TOVR',
 'CR_PROD_CNT_PIL',
 'TURNOVER_CC',
 'TURNOVER_PAYM',
 'AGE',
 'CR_PROD_CNT_CC',
 'REST_DYNAMIC_FDEP_3M',
 'REST_DYNAMIC_IL_1M',
 'CR_PROD_CNT_CCFP',
 'REST_DYNAMIC_CUR_1M',
 'REST_AVG_PAYM',
 'LDEAL_GRACE_DAYS_PCT_MED',
 'REST_DYNAMIC_CUR_3M',
 'TURNOVER_DYNAMIC_CUR_1M',
 'REST_DYNAMIC_PAYM_3M',
 'REST_DYNAMIC_IL_3M',
 'TURNOVER_DYNAMIC_IL_3M',
 'REST_DYNAMIC_PAYM_1M',
 'TURNOVER_DYNAMIC_CUR_3M',
 'PACK',
 'CLNT_SETUP_TENOR',
 'TURNOVER_DYNAMIC_PAYM_3M',
 'TURNOVER_DYNAMIC_PAYM_1M',
 'REST_DYNAMIC_CC_1M',
 'TURNOVER_DYNAMIC_CC_1M',
 'REST_DYNAMIC_CC_3M',
 'TURNOVER_DYNAMIC_CC_3M']

In [68]:
df_test_short = df_test.select(name_test)
df_test_short.show(2, vertical = True, truncate = False)

-RECORD 0--------------------------------------
 _c0                      | 372289             
 ID                       | 519130             
 CR_PROD_CNT_IL           | 0                  
 TURNOVER_DYNAMIC_IL_1M   | 0.0                
 REST_DYNAMIC_FDEP_1M     | 0.0                
 REST_DYNAMIC_SAVE_3M     | 1.0                
 CR_PROD_CNT_VCU          | 0                  
 REST_AVG_CUR             | 13916.8191         
 CR_PROD_CNT_TOVR         | 0                  
 CR_PROD_CNT_PIL          | 0                  
 TURNOVER_CC              | 0.0                
 TURNOVER_PAYM            | 0.0                
 AGE                      | 240                
 CR_PROD_CNT_CC           | 0                  
 REST_DYNAMIC_FDEP_3M     | 0.0                
 REST_DYNAMIC_IL_1M       | 0.0                
 CR_PROD_CNT_CCFP         | 0                  
 REST_DYNAMIC_CUR_1M      | 0.479789155507046  
 REST_AVG_PAYM            | 0.0                
 LDEAL_GRACE_DAYS_PCT_MED | 0.0         

## 2. Удаляем сотавшиеся null

In [13]:
df_train_short.write.options(header='True', delimiter=',').csv("/user/yuriy.perevezentsev/train_short")
df_test_short.write.options(header='True', delimiter=',').csv("/user/yuriy.perevezentsev/test_short")

NameError: name 'df_train_short' is not defined

In [11]:
train_short = spark.read.csv("/user/yuriy.perevezentsev/train_short", header=True, inferSchema=True, sep=',')
# train_short.show(1, vertical = True, truncate = False)

In [12]:
test_short = spark.read.csv("/user/yuriy.perevezentsev/test_short", header=True, inferSchema=True, sep=',')
# test_short.show(1, vertical = True, truncate = False)

In [13]:
train_short = train_short.na.fill(value=-1)

In [14]:
import pyspark.sql.functions as func

for col in train_short.columns:
    train_short = train_short.filter(func.col(col).isNotNull())
train_short.show(1, vertical = True, truncate = False)

-RECORD 0---------------------------------------
 _c0                      | 149375              
 ID                       | 296216              
 CR_PROD_CNT_IL           | 1                   
 TURNOVER_DYNAMIC_IL_1M   | 0.0725595123448051  
 REST_DYNAMIC_FDEP_1M     | 0.0                 
 REST_DYNAMIC_SAVE_3M     | 0.0                 
 CR_PROD_CNT_VCU          | 0                   
 REST_AVG_CUR             | 50625.248           
 CR_PROD_CNT_TOVR         | 0                   
 CR_PROD_CNT_PIL          | 0                   
 TURNOVER_CC              | 0.0                 
 TURNOVER_PAYM            | 0.0                 
 AGE                      | 708                 
 CR_PROD_CNT_CC           | 0                   
 REST_DYNAMIC_FDEP_3M     | 0.0                 
 REST_DYNAMIC_IL_1M       | 0.264638472029086   
 CR_PROD_CNT_CCFP         | 0                   
 REST_DYNAMIC_CUR_1M      | 0.21195876314522002 
 REST_AVG_PAYM            | 0.0                 
 LDEAL_GRACE_DAYS_PC

In [15]:
import pyspark.sql.functions as func

# Проверяем, остлись ли null в train
lst2 = [];
for col in train_short.columns:
    lst2.append(    train_short.filter(func.col(col).isNull()).count()    )
print(sum(lst2))

0


In [16]:
lst2 = [];

# Проверяем, остлись ли null в test

for col in test_short.columns:
    lst2.append(    test_short.filter(func.col(col).isNull()).count()    )
print(sum(lst2))

0


## 3. Работаем со стринговой переменной PACK (StringIndexer)

In [32]:
train_short.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- CR_PROD_CNT_IL: integer (nullable = true)
 |-- TURNOVER_DYNAMIC_IL_1M: double (nullable = true)
 |-- REST_DYNAMIC_FDEP_1M: double (nullable = true)
 |-- REST_DYNAMIC_SAVE_3M: double (nullable = true)
 |-- CR_PROD_CNT_VCU: integer (nullable = true)
 |-- REST_AVG_CUR: double (nullable = true)
 |-- CR_PROD_CNT_TOVR: integer (nullable = true)
 |-- CR_PROD_CNT_PIL: integer (nullable = true)
 |-- TURNOVER_CC: double (nullable = true)
 |-- TURNOVER_PAYM: double (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- CR_PROD_CNT_CC: integer (nullable = true)
 |-- REST_DYNAMIC_FDEP_3M: double (nullable = true)
 |-- REST_DYNAMIC_IL_1M: double (nullable = true)
 |-- CR_PROD_CNT_CCFP: integer (nullable = true)
 |-- REST_DYNAMIC_CUR_1M: double (nullable = true)
 |-- REST_AVG_PAYM: double (nullable = true)
 |-- LDEAL_GRACE_DAYS_PCT_MED: double (nullable = true)
 |-- REST_DYNAMIC_CUR_3M: double (nullable = true)
 |-- 

In [35]:
train_short.select("PACK").distinct().collect()

[Row(PACK='101'),
 Row(PACK='107'),
 Row(PACK='K01'),
 Row(PACK='M01'),
 Row(PACK='104'),
 Row(PACK='102'),
 Row(PACK='103'),
 Row(PACK='108'),
 Row(PACK='105'),
 Row(PACK='O01'),
 Row(PACK='109'),
 Row(PACK='301')]

In [37]:
test_short.select("PACK").distinct().collect()

[Row(PACK='101'),
 Row(PACK='107'),
 Row(PACK='K01'),
 Row(PACK='M01'),
 Row(PACK='104'),
 Row(PACK='102'),
 Row(PACK='103'),
 Row(PACK='105'),
 Row(PACK='O01'),
 Row(PACK='109'),
 Row(PACK='301')]

In [41]:
from pyspark.ml.feature import StringIndexer

qualification_indexer = StringIndexer(inputCol="PACK", outputCol="pack_encoded")

model_indexer = qualification_indexer.fit(train_short)
train_indexed = model_indexer.transform(train_short)
test_indexed = model_indexer.transform(test_short)

In [42]:
train_indexed.select("pack_encoded").distinct().collect()

[Row(pack_encoded=8.0),
 Row(pack_encoded=0.0),
 Row(pack_encoded=7.0),
 Row(pack_encoded=1.0),
 Row(pack_encoded=4.0),
 Row(pack_encoded=11.0),
 Row(pack_encoded=3.0),
 Row(pack_encoded=2.0),
 Row(pack_encoded=10.0),
 Row(pack_encoded=6.0),
 Row(pack_encoded=5.0),
 Row(pack_encoded=9.0)]

In [43]:
test_indexed.select("pack_encoded").distinct().collect()

[Row(pack_encoded=8.0),
 Row(pack_encoded=0.0),
 Row(pack_encoded=7.0),
 Row(pack_encoded=1.0),
 Row(pack_encoded=4.0),
 Row(pack_encoded=3.0),
 Row(pack_encoded=2.0),
 Row(pack_encoded=10.0),
 Row(pack_encoded=6.0),
 Row(pack_encoded=5.0),
 Row(pack_encoded=9.0)]

## 3. Разбиваем на train и validation, VectorAssembler, оцениваем модель

In [44]:
cols = test_indexed.columns
cols.remove("ID")
cols.remove("_c0")
cols.remove("PACK")
cols

['CR_PROD_CNT_IL',
 'TURNOVER_DYNAMIC_IL_1M',
 'REST_DYNAMIC_FDEP_1M',
 'REST_DYNAMIC_SAVE_3M',
 'CR_PROD_CNT_VCU',
 'REST_AVG_CUR',
 'CR_PROD_CNT_TOVR',
 'CR_PROD_CNT_PIL',
 'TURNOVER_CC',
 'TURNOVER_PAYM',
 'AGE',
 'CR_PROD_CNT_CC',
 'REST_DYNAMIC_FDEP_3M',
 'REST_DYNAMIC_IL_1M',
 'CR_PROD_CNT_CCFP',
 'REST_DYNAMIC_CUR_1M',
 'REST_AVG_PAYM',
 'LDEAL_GRACE_DAYS_PCT_MED',
 'REST_DYNAMIC_CUR_3M',
 'TURNOVER_DYNAMIC_CUR_1M',
 'REST_DYNAMIC_PAYM_3M',
 'REST_DYNAMIC_IL_3M',
 'TURNOVER_DYNAMIC_IL_3M',
 'REST_DYNAMIC_PAYM_1M',
 'TURNOVER_DYNAMIC_CUR_3M',
 'CLNT_SETUP_TENOR',
 'TURNOVER_DYNAMIC_PAYM_3M',
 'TURNOVER_DYNAMIC_PAYM_1M',
 'REST_DYNAMIC_CC_1M',
 'TURNOVER_DYNAMIC_CC_1M',
 'REST_DYNAMIC_CC_3M',
 'TURNOVER_DYNAMIC_CC_3M',
 'pack_encoded']

In [45]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = cols, outputCol = "features")

train_indexed1 = assembler.transform(train_indexed)
test_indexed1 = assembler.transform(test_indexed)

In [52]:
# train_indexed1.show(1)

In [49]:
# test_indexed1.show(1)

In [50]:
train = train_indexed1.sampleBy("TARGET", fractions={0: 0.8, 1: 0.8}, seed = 5757)
valid = train_indexed1.join(train, on = "ID", how = "leftanti")

In [53]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(labelCol= "TARGET", maxDepth=4, minInstancesPerNode=3, maxBins=50)

gbt_model = gbt.fit(train)
predictions_valid = gbt_model.transform(valid)

In [54]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol="TARGET", metricName='areaUnderROC')
score = evaluator.evaluate(predictions_valid)
score

0.8057425264140707

In [94]:
test_predictions = gbt_model.transform(test_indexed1)

In [95]:
test_predictions.show(3, vertical = True, truncate = False)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 _c0                      | 372289                                                                                                                                                  
 ID                       | 519130                                                                                                                                                  
 CR_PROD_CNT_IL           | 0                                                                                                                                                       
 TURNOVER_DYNAMIC_IL_1M   | 0.0                                                                                                                                                     
 REST_DYNAMIC_FDEP_1M     | 0.0                                                                

In [96]:
predictions_pd = test_predictions.select(func.col("ID").alias("id"), 
                                         func.col("probability").alias("target")).toPandas()
predictions_pd.head(5)


,id,target
0,519130,"[0.8780515487408743, 0.12194845125912568]"
1,234045,"[0.9475766596354888, 0.05242334036451124]"
2,401256,"[0.9484804763051935, 0.0515195236948065]"
3,551070,"[0.9341360298381903, 0.06586397016180967]"
4,367285,"[0.9411650902609693, 0.058834909739030716]"


In [97]:
# predictions_pd = predictions_pd.sort_values(by=['id'])
predictions_pd['target'] = predictions_pd['target'].apply(lambda x: x[1])
predictions_pd.head(5)

,id,target
0,519130,0.121948
1,234045,0.052423
2,401256,0.051520
3,551070,0.065864
4,367285,0.058835


In [98]:
predictions_pd.to_csv('lab05.csv', index = False, sep ='\t')


In [93]:
# predictions_pd = test_predictions.select("user_id", "item_id", func.col("probability").alias("purchase")).toPandas()
# predictions_pd = predictions_pd.sort_values(by=['user_id', 'item_id'])
# predictions_pd['purchase'] = predictions_pd['purchase'].apply(lambda x: x[1])
# predictions_pd.to_csv('lab03.csv', index=False)

In [75]:
len(predictions_pd.index) # 44399

44399

In [81]:
import pandas as pd
a = pd.read_csv('lab05.csv', index_col=0)

In [80]:
a.head()

,Unnamed: 0,target
id,,
146871,1116,0.262532
146872,6151,0.291358
146894,29800,0.054042
146922,13839,0.317959
146926,7791,0.083821
